In [6]:
import os
print(os.getcwd())

C:\Users\adity\APL


In [8]:
os.makedirs("dataset/spam", exist_ok=True)  # Creates 'spam' folder if it doesn't exist
os.makedirs("dataset/non_spam", exist_ok=True)  # Creates 'non_spam' folder if it doesn't exist


In [14]:
dataset_path = "C:\\Users\\adity\\APL\\dataset"
    spam/         # Folder for spam images
        without_weaving.jpg
        wjoe_17.jpg
        wpfji2y8W3.jpg
    
    non_spam/     # Folder for normal images
        zzz_91064_01f964cdc0_m.jpg
        zzz_9814_3b88fd33e3_m.jpg
        zzz_9953_4bd3547c43_m.jpg


IndentationError: unexpected indent (1958508761.py, line 2)

In [16]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

# Define image transformations (resize, normalize)
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5], std=[0.5])
])

# Set path to dataset
dataset_path = "C:\\Users\\adity\\APL\\dataset"  # Correct dataset path
train_data = datasets.ImageFolder(root=dataset_path, transform=transform)
train_loader = DataLoader(train_data, batch_size=32, shuffle=True)

# Verify class names (should be ['non_spam', 'spam'])
print("Classes found:", train_data.classes)


Classes found: ['non_spam', 'spam']


In [18]:
class SpamCNN(nn.Module):
    def __init__(self):
        super(SpamCNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, stride=1, padding=1)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1)
        self.fc1 = nn.Linear(64 * 56 * 56, 128)
        self.fc2 = nn.Linear(128, 1)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        x = self.pool(torch.relu(self.conv1(x)))
        x = self.pool(torch.relu(self.conv2(x)))
        x = x.view(-1, 64 * 56 * 56)  # Flatten
        x = torch.relu(self.fc1(x))
        x = self.sigmoid(self.fc2(x))
        return x


In [20]:
# Initialize model, loss function, optimizer
model = SpamCNN()
criterion = nn.BCELoss()  # Binary Cross Entropy Loss
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training loop
epochs = 10
for epoch in range(epochs):
    for images, labels in train_loader:
        labels = labels.float().unsqueeze(1)  # Reshape for BCELoss
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
    
    print(f"Epoch {epoch+1}/{epochs}, Loss: {loss.item():.4f}")

# Save the model
torch.save(model.state_dict(), "spam_image_detector.pth")


Epoch 1/10, Loss: 0.6896
Epoch 2/10, Loss: 0.0030
Epoch 3/10, Loss: 0.0000
Epoch 4/10, Loss: 0.0000
Epoch 5/10, Loss: 0.0000
Epoch 6/10, Loss: 0.0000
Epoch 7/10, Loss: 0.0000
Epoch 8/10, Loss: 0.0000
Epoch 9/10, Loss: 0.0000
Epoch 10/10, Loss: 0.0000


In [28]:
# Example: Predict on a new image
from PIL import Image
image_path = r"C:\Users\adity\APL\dataset\non_spam\zzz_91064_01f964cdc0_m.jpg"  # Full path to the image
img = Image.open(image_path)

# Preprocess the image
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5], std=[0.5])
])

img = transform(img).unsqueeze(0)  # Add batch dimension

# Predict
with torch.no_grad():
    prediction = model(img)
    if prediction.item() > 0.5:
        print("Spam Image 🚨")
    else:
        print("Not Spam ✅")


Not Spam ✅
